In [ ]:
!pip install hyperas

In [15]:
import h5py
from os.path import join,exists
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation,Flatten, Bidirectional, LSTM
from tensorflow.keras.optimizers import RMSprop
from hyperas.distributions import choice
from hyperas import optim
from tensorflow.keras.constraints import max_norm
from random import randint
import keras
import numpy as np
from sklearn.utils import shuffle
from hyperopt import Trials, STATUS_OK, tpe
import os
import matplotlib.pyplot as plt

In [3]:
os.chdir('/content/drive/My Drive/github/NNanobody/regression')

In [11]:
def create_model(X_train, Y_train, X_test, Y_test):
    W_maxnorm = 3
    DROPOUT = {{choice([0.3,0.5,0.7])}}
    
    model = Sequential()
    model.add(Bidirectional(LSTM(units=64, return_sequences=True), input_shape=(20,20)))
    model.add(Bidirectional(LSTM(units=64)))
    model.add(Dense(1))

    myoptimizer = RMSprop(lr={{choice([1e-1,0.01,0.001,0.0001,1e-5])}}, rho=0.9, epsilon=1e-6)
    mylossfunc='mean_squared_error'
    model.compile(loss=mylossfunc, optimizer=myoptimizer)
    result = model.fit(X_train, Y_train, batch_size=100, epochs=20,validation_split=0.1, verbose=False)

    val_loss = np.amax(result.history['val_loss'])

    return {'loss': val_loss, 'status': STATUS_OK, 'model': model}

In [12]:
def data():
  data_train = h5py.File('../data/regression/Full Regression/train.h5.batch1','r')
  X_train = np.array(data_train['data']).reshape(-1,20,20)
  Y_train = np.array(data_train['label'])
  X_train_shuffled, Y_train_shuffled = shuffle(X_train, Y_train)

  data_test = h5py.File('../data/regression/Test set Regression/test.h5.batch1', 'r')
  X_test = np.array(data_test['data']).reshape(-1,20,20)
  Y_test = np.array(data_test['label'])
  return X_train_shuffled, Y_train_shuffled, X_test, Y_test

In [16]:
best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='seq_LSTM',
                                          verbose=False)

  0%|          | 0/10 [00:00<?, ?it/s, best loss: ?]

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)



100%|██████████| 10/10 [23:43<00:00, 142.36s/it, best loss: 0.37531405687332153]


In [17]:
X_train, Y_train, X_test, Y_test = data()
best_model.fit(X_train, Y_train, batch_size=100, epochs=20,validation_split=0.1,verbose=True)

Epoch 1/20
610/610 [==============================] - 7s 12ms/step - loss: 0.1482 - val_loss: 0.1713
Epoch 2/20
610/610 [==============================] - 7s 12ms/step - loss: 0.1386 - val_loss: 0.1698
Epoch 3/20
610/610 [==============================] - 7s 12ms/step - loss: 0.1331 - val_loss: 0.1767
Epoch 4/20
610/610 [==============================] - 8s 13ms/step - loss: 0.1275 - val_loss: 0.1887
Epoch 5/20
610/610 [==============================] - 8s 12ms/step - loss: 0.1223 - val_loss: 0.1948
Epoch 6/20
610/610 [==============================] - 7s 12ms/step - loss: 0.1169 - val_loss: 0.2007
Epoch 7/20
610/610 [==============================] - 7s 12ms/step - loss: 0.1126 - val_loss: 0.2068
Epoch 8/20
610/610 [==============================] - 7s 12ms/step - loss: 0.1096 - val_loss: 0.2065
Epoch 9/20
610/610 [==============================] - 7s 12ms/step - loss: 0.1057 - val_loss: 0.2184
Epoch 10/20
610/610 [==============================] - 7s 12ms/step - loss: 0.1023 - val_lo

In [18]:
best_model.evaluate(X_test, Y_test)

710/710 [==============================] - 3s 5ms/step - loss: 0.3621


0.3620619773864746

In [19]:
best_model.save('../weights/regression/Full Regression/seq_LSTM')

INFO:tensorflow:Assets written to: ../weights/regression/Full Regression/seq_LSTM/assets


INFO:tensorflow:Assets written to: ../weights/regression/Full Regression/seq_LSTM/assets
